In [ ]:
########################
# RUN THIS FIRST
########################

# Code used for generating 2 lists of absolute paths; 1 for segmentation files, 1 for labelmap files

# The resulting lists will be fed into the 'process_files' function to copy/rename files and convert segmentation
    # files to labelmaps (retaining 'A' & 'V' labels only)

import os
import glob
import SimpleITK as sitk
import shutil

#Recursion through directories, searches for files based on file name and extension, saves path to each matching file to one of two lists
def find_files_by_extension_and_prefix(root_folder, extension, prefix):
    segmentation_files = []
    final_files = []

    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            
            #S-Segmentation and S-Final (without 'label' in file name) in one list, S-Final with 'label' in file name in other list
            if filename.endswith(extension) and os.path.basename(dirpath).startswith(prefix):
                #".replace(os.path.sep, '/')" is used to corrent the backslashes in Windows' file paths to the Unix forwardslash
                full_path = os.path.join(dirpath, filename).replace(os.path.sep, '/')
                if filename.startswith("S-Segmentation"):
                    segmentation_files.append(full_path)
                elif filename.startswith("S-Final") and "label" not in filename:
                    segmentation_files.append(full_path)
                elif filename.startswith("S-Final") and "label" in filename:
                    final_files.append(full_path)

    return segmentation_files, final_files

#Copy files in segmentation & label lists generated by the 'find_files_by_extension_and_prefix' function
def copy_and_rename_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        for filename in files:
            source_file_path = os.path.join(root, filename)
            parent_dir = os.path.basename(root)
            new_filename = filename + '_' + parent_dir  # Appending parent directory name to the filename
            destination_file_path = os.path.join(destination_dir, new_filename)
            shutil.copy2(source_file_path, destination_file_path)

######################################################################

# Edit this path to point to the desired parent directory to begin searching for segmentation and labelmap files

#root_folder = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done"
root_folder = "Y:/public_resources/RA4-Project/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done"

######################################################################

extension = ".nrrd"
#Specifies directory name prefix to look for - will add '99-' later 
prefix = "88-"

# Creates two lists of absolute paths to each *.nrrd file within the specified parent directory (root_folder)
segmentation_files, final_files = find_files_by_extension_and_prefix(root_folder, extension, prefix)

print("Segmentation files starting with 'S-Segmentation':")
for file in segmentation_files:
    print(file)

print("\nLabelmap files starting with 'S-Final':")
for file in final_files:
    print(file)

In [55]:
# This cell only runs the code that copys segmentation file data to a microsoft word document, can be run after cell 1 is run

import os
import re
#import shutil
import numpy as np
import nrrd
import matplotlib.pyplot as plt
from docx import Document
#from termcolor import colored

# Function only takes in list of absolute paths to the segmentation files
def process_files(files):

    # Loops through each absolute path in the segmentation files list
    for file_path in files:
        
        # Gets header data from current segmentation file
        header = nrrd.read_header(file_path)
        
        # Initialize segment data for the current file
        segments = []
        segment_count = 0
        
        # Extracts and process segment data
        for key, value in header.items():
            if key.startswith('Segment') and '_Name' in key:
                
                # Used to remove the instances of '0' getting printed with the segmentation label names
                if value != '0':
                    segment_name = value#.split()[0]  # Get only the first part of the value
                elif value == '0':
                    continue

                # Gets the number associated with the Segment#_Color, where # is some number
                # Used to build the segment color key to find each color rgb value
                segment_number = key.split('_')[0].replace('Segment', '')
                segment_color_key = f'Segment{segment_number}_Color'
                
                # Extracts segment color rgb value, then encases the string in brackets
                segment_color = "".join(str(v) for v in header.get(segment_color_key, ''))
                segment_color = '[' + segment_color + ']'

                # Renames segment color based on values associated with the key
                # Other colors will not be renamed and will be reported as their rgb values
                if segment_color == '[0.847059 0.396078 0.309804]':                   
                    segment_color = 'red'
                elif segment_color == '[0 0.592157 0.807843]':
                    segment_color = 'blue'
                elif segment_color == '[0.901961 0.862745 0.27451]':
                    segment_color = 'yellow'

                # Appends segment label info one at a time to the 'segments list'
                # Example: 'Name: A Color: red'
                segments.append({'Name': segment_name, 'Color': segment_color})
                segment_count += 1

        # Initializes variable for the /Case-~ filename below
        extracted_substring = ''

        # Check if the file path starts with '/Case-' and ends with '.nrrd'
        pattern = r'/Case-(.*?\.nrrd)'
        match = re.search(pattern, file_path)
        if match:
            # Extracts the substring between (and including) '/Case-' and '.nrrd'
            extracted_substring = ('/Case-' + match.group(1))

        # Appends file data to the list
        file_data_list.append({
            #'File': file_path,
            'File': extracted_substring,
            'Segments': segments,
            'SegmentCount': segment_count
        })

        # Adds file data to the table
        for file_data in file_data_list:
            row_cells = table.add_row().cells
            row_cells[0].text = file_data['File']
            row_cells[3].text = str(file_data['SegmentCount'])
            
            segment_names = '\n'.join(seg['Name'] for seg in file_data['Segments'])
            row_cells[1].text = segment_names
            
            segment_colors = '\n'.join(seg['Color'] for seg in file_data['Segments'])

            row_cells[2].text = segment_colors

        # Save the document
        doc.save('segment_data.docx')

        # Clear 'file_data_list' for the next loop
        file_data_list.pop()

########################################################3

# Creates a Word document and table heading
doc = Document()
doc.add_heading('Segment Data', level=1)

# Initializes a list to store file data
file_data_list = []

# Creates a table and add header row
table = doc.add_table(rows=1, cols=4)
table.style = 'Table Grid'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'File'
hdr_cells[1].text = 'Segment Names'
hdr_cells[2].text = 'Segment Colors'
hdr_cells[3].text = 'Segment Count'

# Changes working directory
#os.chdir('path/to/dir')

process_files(segmentation_files)